In [95]:
import jieba
from jieba import posseg

## POS => Part of Speech(名词、动词...)

In [96]:
text = """
12月24日，佟丽娅在绿洲更新圣诞美照，并配文：“平安夜快乐，大家吃到甜甜的苹果了吗？” 照片中的她手拿平安果，身穿温暖浅色系毛衣，一脸温婉笑容直视镜头，优雅美好，气质满分。
"""

In [97]:
for w in posseg.cut(text):
    print(w) 


/x
12/m
月/m
24/m
日/m
，/x
佟丽娅/nr
在/p
绿洲/n
更新/d
圣诞/t
美照/ns
，/x
并配/v
文/n
：/x
“/x
平安夜/nr
快乐/a
，/x
大家/n
吃/v
到/v
甜甜的/z
苹果/n
了/ul
吗/y
？/x
”/x
 /x
照片/n
中/f
的/uj
她/r
手/n
拿/v
平安/a
果/ng
，/x
身穿/v
温暖/an
浅色系/n
毛衣/n
，/x
一脸/m
温婉/a
笑容/n
直视/v
镜头/n
，/x
优雅/a
美好/a
，/x
气质/n
满分/d
。/x

/x


## Define a function. Input is a word, output is some similar words.

In [98]:
csv_path = "D:\\new开课吧人工智能nlp\\datasource\\sqlResult_1558435.csv"

In [99]:
import pandas as pd

In [100]:
content = pd.read_csv(csv_path, encoding='gb18030')
content = content.fillna('')
news_content = content['content'].tolist()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xba in position 0: invalid start byte

In [ ]:
cut_file = open("D:\\new开课吧人工智能nlp\\jupyters_and_slides\\2019-summer\\article_9k_cut.txt", 'w')

In [ ]:
def cut(string): return ' '.join(jieba.cut(string))

In [ ]:
cut('这是一个测试')

import re

def token(string):
    return re.findall(r'[\d|\w]+', string)

token('这是一个测试\n\n\n')

In [ ]:
news_content = [token(n) for n in news_content]
news_content = [' '.join(n) for n in news_content]
news_content = [cut(n) for n in news_content]
news_content[1]


In [ ]:
with open('news_sentences_cut.txt', 'w') as f:
    for n in news_content:
        f.write(n + '\n')

In [ ]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [ ]:
news_word2ve = Word2Vec(LineSentence('news_sentences_cut.txt'), size=35, workers=8)
news_word2ve.most_similar('葡萄牙', topn=20)

## More Data, More Results
    1. 分词的问题
    2. *数据量*，数据越多，效果越好，维基百科加进来，那么同义词就要好很多

In [ ]:
news_word2ve.most_similar('捷克', topn=20)
news_word2ve.most_similar('说', topn=30)
news_word2ve.most_similar('认为', topn=30)
news_word2ve.most_similar('建议', topn=10)

In [ ]:
from collections import defaultdict

def get_related_words(initial_words, model):
    """
    :param initial_words: they are initial words we already know 
    :param model: the word2vec model
    :return: 
    """
    unseen = initial_words
    seen = defaultdict(int)
    max_size = 500 # could be greater
    
    while unseen and len(seen) < max_size:
        if len(seen) % 50 == 0:
            print('seen length: {} '.format(len(seen)))
        
        node = unseen.pop(0)
        new_expanding = [w for w, s in model.most_similar(node, topn=20)]
        
        unseen += new_expanding
        seen[node] += 1
        
        # optional 1: 1.score function could be revised
        # optional 2: 2.using dynamic programming to reduce computing time 
        
    return seen
    

In [ ]:
len(news_word2ve.wv.vocab)

In [ ]:
related_words = get_related_words(['说', '表示'], news_word2ve)

In [ ]:
sorted(related_words.items(), key=lambda x: x[1], reversed=True)